In [1]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset,DataLoader
from feature_generator import Yelp_Dataset
from model_pos_attention import Classifier, BiLSTM, SelfAttentiveEncoder

In [2]:
cuda = torch.device('cuda:1')

In [16]:
model = torch.load('/scratch/near/anlp/saved_model_pos_attention_find_tune/2/epoch_8_0.78model')
model = model.to(cuda)

In [17]:
test_loader = DataLoader(Yelp_Dataset('test',cuda),batch_size=50)

In [18]:
model.eval()
total_correct=0
l = []
p = []
for word_id,pos_id,label in test_loader:
    hidden = model.init_hidden(50)
    pred, attention,a2 = model(word_id,pos_id,hidden)
    pred = torch.max(pred, 1)[1]
    p.append(pred.cpu().detach().numpy())
    l.append(label.cpu().detach().numpy())
    total_correct += torch.sum((pred == label).float())
print(total_correct.data / 2000)

tensor(0.6810, device='cuda:1')


In [19]:
dev_loader = DataLoader(Yelp_Dataset('dev',cuda),batch_size=50)

In [20]:
model.eval()
total_correct=0
for word_id,pos_id,label in dev_loader:
    hidden = model.init_hidden(50)
    pred, attention,a2 = model(word_id,pos_id,hidden)
    pred = torch.max(pred, 1)[1]
    total_correct += torch.sum((pred == label).float())
print(total_correct.data / 2000)

tensor(0.6755, device='cuda:1')


In [15]:
# result for the test set
# word2vec only 0.6860
# word2vec only fine tune 0.7150
# word2vec+pos2vec 0.6600
# word2vec+pos2vec fine tune 0.7025
# word2vec + pos one_hot 0.6950


# word only scratch embedding: 0.7305
# pos scratch embedding 0.7205

# pos2vec double attention 0.6920
# pos one_hot double attention 0.682
# pos scratch embedding double attention 0.7305

# word2vec one hot fine tune 69.6
# word2vec pos2vec attention fine tune: 68.1
# word2vec one hot attention fine tune: 56.1

In [21]:
# result for the dev set
# word2vec only 0.6660
# word2vec only fine tune 0.7005
# word2vec+pos2vec 0.6535
# word2vec+pos2vec fine tune 0.6930
# word2vec + pos one_hot 0.6720

# word only scratch embedding: 0.7300
# pos scratch embedding 0.7195

# pos2vec double attention 0.6845
# pos one_hot double attention 0.6585
# pos scratch embedding double attention  0.7350

# word2vec one hot fine tune 69.6
# word2vec pos2vec attention fine tune: 67.55
# word2vec one hot attention fine tune: 55.85

In [ ]:
ll = []
pp = []
for ele in l:
    for x in ele:
        ll.append(x)
for ele in p:
    for x in ele:
        pp.append(x)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    
#     fig = plt.figure(figsize=(7,7))
#     ax = fig.add_subplot(111)
    
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

plot_confusion_matrix(ll,pp, classes=np.array([1,2,3,4,5]),
                      title='Confusion Matrix of Star Predictions')

In [ ]:
word2vec = torch.load('/scratch/near/anlp/word2vec_matrix.pt')

In [ ]:
word2vec.shape

In [ ]:
pos2vec = torch.load('/scratch/near/anlp/pos2vec_matrix.pt')

In [ ]:
pos_dict = json.load(open('/scratch/near/anlp/pos_dictionary.json'))

In [ ]:
pos_dict['NN']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cs = []
for i in range(pos2vec.shape[0]):
    cos = cosine_similarity(pos2vec[3].reshape(1,-1),pos2vec[i].reshape(1,-1))
    cs.append((cos,i))

In [ ]:
cs.sort()

In [ ]:
result = list(reversed(cs))

In [ ]:
d = {pos_dict[p]:p for p in pos_dict}

In [ ]:
for i in result:
    print(i[0],d[i[1]])